In [2]:
import pandas as pd

filteredDf = pd.read_csv('attribute_list_filtered.csv')
filteredDf.head()

,file_name,hair_color,eyeglasses,smiling,young,human
0,1,1,-1,1,1,-1
1,2,4,-1,1,1,1
2,3,5,-1,1,-1,-1
3,7,2,-1,1,1,-1
4,8,3,-1,1,1,-1


In [3]:
from sklearn.preprocessing import OneHotEncoder

#one-hot-enconding:
def get_one_hot():
    df = pd.read_csv('attribute_list_filtered.csv')
    fileNameDf = df['file_name']
    df = df.drop('file_name', 1)
    enc = OneHotEncoder(categories='auto')
    enc.fit(df)
    onehotlabels = enc.transform(df).toarray()
    print(enc.get_feature_names())
    print(onehotlabels.shape)
    oneHotCols = ["hair_-1", "hair_0", "hair_1", "hair_2", "hair_3", "hair_4",
              "hair_5", "eyeglasses_-1", "eyeglasses_1", "smiling_-1", "smiling_1", 
              "young_-1", "young_1", "human_-1", "human_1"]
    oneHotDf = pd.DataFrame(onehotlabels, columns = oneHotCols)
    oneHotDf = pd.concat([fileNameDf, oneHotDf], axis=1)
    oneHotDf.to_csv('attribute_list_filtered_onehot.csv', index=False)
    return oneHotDf

oneHotDf = get_one_hot()

['x0_-1' 'x0_0' 'x0_1' 'x0_2' 'x0_3' 'x0_4' 'x0_5' 'x1_-1' 'x1_1' 'x2_-1'
 'x2_1' 'x3_-1' 'x3_1' 'x4_-1' 'x4_1']
(4565, 15)


In [4]:
len(oneHotDf)

4565

In [5]:
#data augmentation to prevent over-fitting example:
'''https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html'''
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

def generate_transformed_images(image_path,
                                output_path='transformed',
                                rotiation_range=40, 
                                width_shift_range=0.2, 
                                height_shift_range=0.2, 
                                shear_range=0.2, 
                                zoom_range=0.2, 
                                horizontal_flip=True,
                                fill_mode='nearest'):

    datagen = ImageDataGenerator(
            rotation_range=rotation_range,
            width_shift_range=width_shift_range,
            height_shift_range=height_shift_range,
            shear_range=shear_range,
            zoom_range=zoom_range,
            horizontal_flip=horizontal_flip,
            fill_mode=fill_mode)

    img = load_img(image_path)
    img_arr = img_to_array(img) #shape: (3, 256, 256)
    img_arr = img_arr.reshape((1,)+img_arr.shape) #adds extra dimension, shape: (1,3,256,256)

    #generate batches of randomly transformed images, save in transformed dir
    if os.path.isdir(output_path)==False:
        os.mkdir(output_path)

    i = 0
    flow = datagen.flow(img_arr, batch_size=1,
                        save_to_dir=output_path, 
                        save_prefix=os.path.basename(image_path), 
                        save_format='png')
    for batch in flow:
        i+=1
        if i>20:
            break

In [7]:
#splitting into train, validation, and test sets:
import numpy as np
def split_dataset_random(df, validation_split=0.2, test_split=0.2, seed=0):
    np.random.seed(seed) #if the same seed is used the same random split will be produced
    train_split = 1 - validation_split - test_split
    indeces_array = [int(train_split*len(df)), int((1-test_split)*len(df))]
    '''example, [2, 3] would, for axis=0, result in

        ary[:2]
        ary[2:3]
        ary[3:]

    '''
    train, validate, test = np.split(df.sample(frac=1), indeces_array)
    return train, validate, test

#train, validate, test = split_dataset_random(oneHotDf, seed=123)
#print("train length: ", len(train), "| ratio: ", len(train)/len(oneHotDf))
#print("validate length: ", len(validate), "| ratio: ", len(validate)/len(oneHotDf))
#print("test length: ", len(test), "| ratio: ", len(test)/len(oneHotDf))

In [8]:
#Move images to train, test, validate directories according to split:
import shutil
import os

def move_split_images(train, validate, test):
    imagesPath = './images/'
    trainPath = './images/train/'
    validatePath = './images/validate/'
    testPath = './images/test/'
    newPaths = [trainPath, validatePath, testPath]
    dfList = [train, validate, test]
    for path in newPaths:
        if os.path.isdir(path)==False:
            os.mkdir(path)
    
    for df in dfList:
        for idx, row in df.iterrows():
            fileName = str(int(row['file_name']))+'.png'
            filePath = os.path.join(imagesPath, fileName)
            trainFilePath = os.path.join(trainPath, fileName)
            validateFilePath = os.path.join(validatePath, fileName)
            testFilePath = os.path.join(testPath, fileName)
            if(df.equals(test)):
                newFilePath = testFilePath
            if(df.equals(train)):
                newFilePath = trainFilePath
            if(df.equals(validate)):
                newFilePath = validateFilePath
            if os.path.isfile(filePath):
                shutil.move(filePath, newFilePath)
            elif os.path.isfile(testFilePath):
                shutil.move(testFilePath, newFilePath)
            elif os.path.isfile(validateFilePath):
                shutil.move(validateFilePath, newFilePath)
            elif os.path.isfile(trainFilePath):
                shutil.move(trainFilePath, newFilePath)
            else:
                print("File missing: ", fileName)

In [44]:
import shutil
import os

def reset_images_to_main_dir():
    dest_dir = './images/'
    walker = os.walk(dest_dir)
    rem_dirs = walker.__next__()[1]
 
    for data in walker:
        for files in data[2]:
            try:
                shutil.move(data[0] + os.sep + files, dest_dir)
            except shutil.Error:
                print(shutil.Error)
                continue

def move_classes_images(train, validate, test):
    reset_images_to_main_dir()
    imagesPath = './images/'
    trainPath = './images/train/'
    validatePath = './images/validate/'
    testPath = './images/test/'
    newPaths = [trainPath, validatePath, testPath]
    dfList = [train, validate, test]
    for path in newPaths:
        if os.path.isdir(path)==False:
            os.mkdir(path)
    
    for df in dfList:
        for idx, row in df.iterrows():
            if int(row['human'])==-1:
                pathAppend = 'cartoon/'
            elif int(row['human']==1):
                pathAppend = 'human/'
            fileName = str(int(row['file_name']))+'.png'
            filePath = os.path.join(imagesPath, fileName)
            trainClassPath = os.path.join(trainPath, pathAppend)
            trainFilePath = os.path.join(trainClassPath, fileName)
            validateClassPath = os.path.join(validatePath, pathAppend)
            validateFilePath = os.path.join(validateClassPath, fileName)
            testClassPath = os.path.join(testPath, pathAppend)
            testFilePath = os.path.join(testClassPath, fileName)
            classPaths = [trainClassPath, validateClassPath, testClassPath]
            for path in classPaths:
                if os.path.isdir(path)==False:
                    os.mkdir(path)
            if(df.equals(test)):
                newFilePath = testFilePath
            if(df.equals(train)):
                newFilePath = trainFilePath
            if(df.equals(validate)):
                newFilePath = validateFilePath
            if os.path.isfile(filePath):
                shutil.move(filePath, newFilePath)
            elif os.path.isfile(testFilePath):
                shutil.move(testFilePath, newFilePath)
            elif os.path.isfile(validateFilePath):
                shutil.move(validateFilePath, newFilePath)
            elif os.path.isfile(trainFilePath):
                shutil.move(trainFilePath, newFilePath)
            else:
                print("File missing: ", fileName)

In [27]:
#validatePath = './images/validate/'
#labels_cols = list(train)[1:]
#print(labels_cols)

In [49]:
def get_data_split(df=oneHotDf, seed=123):
    train, validate, test = split_dataset_random(df, seed=111)
    print("train length: ", len(train), "| ratio: ", len(train)/len(filteredDf))
    print("validate length: ", len(validate), "| ratio: ", len(validate)/len(filteredDf))
    print("test length: ", len(test), "| ratio: ", len(test)/len(filteredDf))

    move_split_images(train, validate, test)
    
    return train, validate, test

train, validate, test = get_data_split(df=filteredDf, seed=0)
%store train
%store validate
%store test

train length:  2739 | ratio:  0.6
validate length:  913 | ratio:  0.2
test length:  913 | ratio:  0.2
Stored 'train' (DataFrame)
Stored 'validate' (DataFrame)
Stored 'test' (DataFrame)


In [ ]:
def move_classes_images(train, validate, test):
    reset_images_to_main_dir()
    imagesPath = './images/'
    trainPath = './images/train/'
    validatePath = './images/validate/'
    testPath = './images/test/'
    newPaths = [trainPath, validatePath, testPath]
    dfList = [train, validate, test]
    for path in newPaths:
        if os.path.isdir(path)==False:
            os.mkdir(path)
    
    for df in dfList:
        for idx, row in df.iterrows():
            if int(row['human'])==-1:
                pathAppend = 'cartoon/'
            elif int(row['human']==1):
                pathAppend = 'human/'
            fileName = str(int(row['file_name']))+'.png'
            filePath = os.path.join(imagesPath, fileName)
            trainClassPath = os.path.join(trainPath, pathAppend)
            trainFilePath = os.path.join(trainClassPath, fileName)
            validateClassPath = os.path.join(validatePath, pathAppend)
            validateFilePath = os.path.join(validateClassPath, fileName)
            testClassPath = os.path.join(testPath, pathAppend)
            testFilePath = os.path.join(testClassPath, fileName)
            classPaths = [trainClassPath, validateClassPath, testClassPath]
            for path in classPaths:
                if os.path.isdir(path)==False:
                    os.mkdir(path)
            if(df.equals(test)):
                newFilePath = testFilePath
            if(df.equals(train)):
                newFilePath = trainFilePath
            if(df.equals(validate)):
                newFilePath = validateFilePath
            if os.path.isfile(filePath):
                shutil.move(filePath, newFilePath)
            elif os.path.isfile(testFilePath):
                shutil.move(testFilePath, newFilePath)
            elif os.path.isfile(validateFilePath):
                shutil.move(validateFilePath, newFilePath)
            elif os.path.isfile(trainFilePath):
                shutil.move(trainFilePath, newFilePath)
            else:
                print("File missing: ", fileName)

In [48]:
reset_images_to_main_dir()